In [1]:
import numpy as np

### Normal forward mode with tape used for adjoint

In [56]:
def forward(x0 = np.array([1, 3, 3])):
    
    x = np.zeros(3)
    tape = []
    for i in range(3):
        x[i] = x0[i]

    y = x[0]**2
    
    tape.append(np.copy(x))

    for i in range(3):

        x[i] = y + x[i]**2 

    # Convective adjustment

    for i in range(2):

        tape.append(np.copy(x))

        if (x[i] < x[i+1]):
            x[i] = 0.5*(x[i] + x[i+1])
            x[i+1] = x[i]

    tape.append(np.copy(x))

    fc = (x[0]-5.5)**2 + 2.*x[1] + 3.*x[2]

    return fc, tape

forward()

(40.25,
 [array([1., 3., 3.]),
  array([ 2., 10., 10.]),
  array([ 6.,  6., 10.]),
  array([6., 8., 8.])])

In [34]:
(forward(x0 = np.array([1.001, 3, 3]))[0] - forward(x0 = np.array([1, 3, 3]))[0])/0.001

15.516759002245806

In [35]:
(forward(x0 = np.array([1, 3.001, 3]))[0] - forward(x0 = np.array([1, 3, 3]))[0])/0.001

10.510753000247064

In [36]:
(forward(x0 = np.array([1, 3, 3.001]))[0] - forward(x0 = np.array([1, 3, 3]))[0])/0.001

15.002499999994257

### Tangent linear mode

In [31]:
def forward_d(x0 = np.array([1, 3, 3]), x0d = [1, 0, 0]):

    x = np.zeros(3)
    xd = np.zeros(3)
    

    
    for i in range(3):
        
        # [xd[i]]  = [0. 1.] [xd]
        # [x0d[i]] = [0. 1.] [x0d[i]] 
    
        xd[i] = x0d[i]
        x[i] = x0[i]

    # [yd]    = [0. 2*x[0]] [yd]
    # [xd[0]] = [0.     1.] [xd[0]] 
    yd = 2*x[0]*xd[0]
    y = x[0]**2
    
    for i in range(3):

        # [xd[i]]  = [2*x[i]  1.] [xd[i]]
        # [yd]     = [0.      1.] [yd]
        
        xd[i] = yd + 2*x[i]*xd[i]
        x[i] = y + x[i]**2 
        
    # Convective adjustment

    for i in range(2):

        if (x[i] < x[i+1]):
            
            # [xd[i]]   = [0.5 0.5] [xd[i]]
            # [xd[i+1]] = [0.   1.] [xd[i+1]]
        
            xd[i] = 0.5*(xd[i] + xd[i+1])
            x[i] = 0.5*(x[i] + x[i+1])

            # [xd[i]]   = [1. 0.] [xd[i]]
            # [xd[i+1]] = [1. 0.] [xd[i+1]]
            
            xd[i+1] = xd[i]
            x[i+1] = x[i]

    # [xd[0]] = [1.           0. 0. 0.] [xd[0]]
    # [xd[1]] = [0.           1. 0. 0.] [xd[1]]
    # [xd[2]] = [0.           0. 1. 0.] [xd[2]]
    # [fcd]   = [2*(x[0]-5.5) 2. 3. 0.] [fcd]
    fcd = 2*(x[0]-5.5)*xd[0] + 2.*xd[1] + 3.*xd[2]
    fc = (x[0]-5.5)**2 + 2.*x[1] + 3.*x[2]


    return fcd

forward_d(x0d = [1, 0, 0]), forward_d(x0d = [0, 1, 0]), forward_d(x0d = [0, 0, 1])

(15.5, 10.5, 15.0)

In [65]:
_, tape = forward(x0 = np.array([1, 3, 3]))

def forward_b(x0 = np.array([1, 3, 3]), tape = tape):
    
    xb = np.zeros(3)
    x0b = np.zeros(3)
    yb = 0.0
    
    fcb = 1.0
    
    # [xb[0]] = [1. 0. 0. 2*(x[0]-5.5)] [xb[0]]
    # [xb[1]] = [0. 1. 0. 2.          ] [xb[1]]
    # [xb[2]] = [0. 0. 1. 3.          ] [xb[2]]
    # [fcb]   = [0. 0. 0. 0.          ] [fcb] 
    
    x = tape.pop()
    
    xb[0] = xb[0] +  2*(x[0]-5.5) * fcb
    xb[1] = xb[1] +  2.           * fcb
    xb[2] = xb[2] +  3.           * fcb
    
    for i in range(1,-1,-1):
        
        x = tape.pop()
        
        if (x[i] < x[i+1]):
        
            # [xb[i]]   = [1. 1.] [xb[i]]
            # [xb[i+1]] = [0. 0.] [xb[i+1]]

            xb[i] = xb[i] + xb[i+1]
            xb[i+1] = 0.

            # [xb[i]]   = [0.5 0.] [xb[i]]
            # [xb[i+1]] = [0.5 1.] [xb[i+1]]   

            xb[i+1] = 0.5*xb[i] + xb[i+1]
            xb[i]   = 0.5*xb[i]
            
    x = tape.pop()
    
    for i in range(2,-1,-1):

        # [xb[i]]  = [2*x[i] 0.] [xb[i]]
        # [yb]     = [1.      1.] [yb] 
        
        yb = yb + xb[i]
        xb[i] = 2*x[i]*xb[i]
        
    # [yb]    = [0.     0.] [yb]
    # [xb[0]] = [2*x[0] 1.] [xb[0]] 
    
    xb[0] = xb[0] + 2*x[0]*yb
    yb    = 0.0

    for i in range(2,-1,-1):
        
        # [xb[i]]  = [0. 0.] [xb]
        # [x0b[i]] = [1. 1.] [x0b[i]] 
    
        x0b[i] = x0b[i] + xb[i]
        xb[i]  = 0.0
        
    return x0b

In [67]:
_, tape = forward(x0 = np.array([1, 3, 3]))
forward_b(x0 = np.array([1, 3, 3]), tape = tape)

array([15.5, 10.5, 15. ])